Ed - "I found a girrrl"

Me - *faints, collapses, rolls on the floor, foams in the mouth

In [1]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

In [4]:
current_dir = os.path.dirname(os.path.abspath("Ed Sheeran"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

config.read(config_path)

client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [3]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [6]:
def collect_spotify_data():
    start_time = timer()
    data = []

    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist:Ed Sheeran', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5) 
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break 

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print(f"Elapsed time:", timekeeper(elapsed_time))
    return df_raw

df_spotify = collect_spotify_data()

Elapsed time: 4 minutes, 24 seconds


In [16]:
ed = df_spotify[df_spotify['artist_name'] == 'Ed Sheeran']
df_spotify.to_csv('raw material')

In [17]:
ed

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Ed Sheeran,Shivers,50nfwKoDiSYg8zOCREWAm5,=,32iAEBstCjauDhyKpGjTuq,2021-10-25,82,False,0.788,207853,...,-2.724,1,0.0856,0.2810,0.000000,0.0424,0.822,141.020,4,[]
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,85,False,0.599,263400,...,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050,3,[]
2,Ed Sheeran,Shape of You,7qiZfU4dY1lWllzX7mPBI3,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,85,False,0.825,233713,...,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,4,[]
3,Ed Sheeran,Bad Habits,3rmo8F54jFF8OgYsqTxm5d,=,32iAEBstCjauDhyKpGjTuq,2021-10-25,80,False,0.807,230747,...,-3.745,0,0.0347,0.0451,0.000028,0.3660,0.537,126.011,4,[]
4,Ed Sheeran,Castle on the Hill,6PCUP3dWmTjcTtXY02oFdT,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,78,False,0.461,261154,...,-4.868,1,0.0989,0.0232,0.000011,0.1400,0.471,135.007,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ed Sheeran,Perfect - Acoustic,71BidKYZj9UJesSPXX21FG,Acoustic Vibes,24TUp6UvuBpcWD1dimGe2y,2022-08-19,1,False,0.465,260867,...,-5.790,1,0.0390,0.4590,0.000000,0.1140,0.407,189.948,3,[]
996,Ed Sheeran,Happier - Acoustic,4YmKuKHV61jBSNwnVKc1DF,Brunch,5eR0rvH19aAuXlLqHQ9qTl,2023-11-11,0,False,0.501,206209,...,-8.117,1,0.0258,0.0417,0.000002,0.1330,0.151,91.007,4,[]
997,Ed Sheeran,Galway Girl,2dYC9D9FJ0zotF288AQ8pe,After Ski 2021,5TwXV5sbz94vuCXjcASpdA,2021-11-05,0,False,0.599,168253,...,-5.882,1,0.0846,0.0783,0.000000,0.4760,0.809,100.192,4,[]
998,Ed Sheeran,Celestial,4LZUZlsVati8ByYpQhm5ID,Happy Mood,54QwsbbIcgk6kQcoROpJU7,2023-06-15,1,False,0.574,209027,...,-1.248,1,0.0397,0.0529,0.000000,0.1610,0.501,123.038,4,[]
